# Retrieval Augmented Generation


## Setup


In [3]:
%pip install -U llama-index
%pip install -U llama-index-embeddings-huggingface
%pip install -U pinecone
%pip install -U llama-index-vector-stores-pinecone
%pip install -U deepeval
%pip install -U llama-index-llms-anthropic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 862.7/862.7 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 23.8 MB/s eta 0:00:00


In [3]:
import os

from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.anthropic import Anthropic
from llama_index.core import VectorStoreIndex
import numpy as np
import json
from llama_index.core.schema import TextNode
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.vector_stores.pinecone import PineconeVectorStore
from pinecone import Pinecone, ServerlessSpec
from google.colab import userdata

PINECONE_API_KEY = userdata.get("PINECONE_API_KEY")
GITHUB_TOKEN = userdata.get('GITHUB_TOKEN')
ANTHROPIC_API_KEY = userdata.get("ANTHROPIC_API_KEY")

In [5]:
!git clone https://$GITHUB_TOKEN@github.com/J-Gann/medfluencer.git

Cloning into 'medfluencer'...
remote: Enumerating objects: 89, done.
remote: Counting objects: 100% (89/89), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 89 (delta 30), reused 65 (delta 12), pack-reused 0
Receiving objects: 100% (89/89), 9.06 MiB | 6.84 MiB/s, done.
Resolving deltas: 100% (30/30), done.
Filtering content: 100% (27/27), 13.51 GiB | 33.31 MiB/s, done.


In [4]:
embed_model = HuggingFaceEmbedding(
    model_name="T-Systems-onsite/cross-en-de-roberta-sentence-transformer"
)

In [5]:
llm = Anthropic(
    model="claude-3-5-sonnet-20240620",
    api_key=ANTHROPIC_API_KEY,
    max_tokens=4096,
)

In [6]:
pc = Pinecone(api_key=PINECONE_API_KEY)

pinecone_index = pc.Index("medfluencer-videos-index-t-systems")
vector_store = PineconeVectorStore(pinecone_index=pinecone_index)
index_videos = VectorStoreIndex.from_vector_store(vector_store, embed_model=embed_model)

pinecone_index = pc.Index("medfluencer-comments-index-t-systems")
vector_store = PineconeVectorStore(pinecone_index=pinecone_index)
index_comments = VectorStoreIndex.from_vector_store(
    vector_store, embed_model=embed_model
)

#### Retriever


In [7]:
retriever_videos = VectorIndexRetriever(
    index=index_videos,
    similarity_top_k=20,
)

In [8]:
retriever_comments = VectorIndexRetriever(
    index=index_comments,
    similarity_top_k=20,
)

### Reranking


In [9]:
from llama_index.core.postprocessor import SentenceTransformerRerank

rerank = SentenceTransformerRerank(
    model="cross-encoder/msmarco-MiniLM-L6-en-de-v1", top_n=5
)

### Query Engine


In [10]:
from llama_index.core import get_response_synthesizer
from llama_index.core.query_engine import RetrieverQueryEngine

In [11]:
response_synthesizer = get_response_synthesizer(llm=llm)

query_engine_videos = RetrieverQueryEngine(
    retriever=retriever_videos,
    response_synthesizer=response_synthesizer,
    node_postprocessors=[rerank],
)

query_engine_comments = RetrieverQueryEngine(
    retriever=retriever_comments,
    response_synthesizer=response_synthesizer,
    node_postprocessors=[rerank],
)

# Evaluation


In [12]:
import json

#### Generate questions


In [13]:
with open("medfluencer/medical_fields.json", "r", encoding="UTF-8") as file:
    medical_fields = json.load(file)

In [14]:
from collections import defaultdict

In [ ]:
# Generates questions for each medical field

# field_questions = defaultdict(list)
# num_questions = 5
# for field in medical_fields:
#    for _ in range(num_questions):
#        question = llm.complete(
#            f"Generiere eine medizinische Frage eines Laiens, die medizinischem Personal (Arthelferin, Arzt) typischerweise in folgenden Fachbereich gestellt wird: {field}\n Antworte ausschließlich mit der Frage!"
#        ).text
#        field_questions[field].append(question)
# with open("evaluation/questions_per_field.json", "w") as file:
#    json.dump(field_questions, file)

In [15]:
with open("medfluencer/evaluation/questions_per_field.json", "r") as file:
    field_questions = json.load(file)

In [ ]:
# answers_per_field = defaultdict(list)
# for field, questions in field_questions.items():
#    for question in questions:
#        res = query_engine.query(question)
#        answer = res.response
#        context = [node.text for node in res.source_nodes]
#        answers_per_field[field].append((answer, context))
# with open("evaluation/answers_per_field.json", "w") as file:
#    json.dump(answers_per_field, file)

In [16]:
with open("medfluencer/evaluation/answers_per_field.json", "r") as file:
    answers_per_field = json.load(file)

#### DeepEval


In [17]:
from deepeval import evaluate
from deepeval.metrics import (
    AnswerRelevancyMetric,
    FaithfulnessMetric,
    ContextualRelevancyMetric,
    HallucinationMetric,
)
from deepeval.test_case import LLMTestCase

In [18]:
test_cases_per_field = defaultdict(list)

for field, questions in field_questions.items():
    for idx, question in enumerate(questions):
        actual_output, retrieval_context = answers_per_field[field][idx]
        test_case = LLMTestCase(
            input=question,
            actual_output=actual_output,
            retrieval_context=retrieval_context,
            context=retrieval_context,
        )

        test_cases_per_field[field].append(test_case)

In [19]:
len(list(test_cases_per_field.keys())) * 10 * 4

2520

In [20]:
from deepeval.models.base_model import DeepEvalBaseLLM

class Claude(DeepEvalBaseLLM):
    def __init__(
        self
    ):
        self.model = llm = Anthropic(
            model="claude-3-5-sonnet-20240620",
            api_key=ANTHROPIC_API_KEY,
            max_tokens=4096,
        )

    def load_model(self):
        return self.model

    def generate(self, prompt: str) -> str:
        model = self.load_model()

        res = model.complete(prompt)
        return res.text

    async def a_generate(self, prompt: str) -> str:
        return self.generate(prompt)

    def get_model_name(self):
        return "claude-3-5-sonnet-20240620"

In [22]:
model = Claude()

answer_relevancy_metric = AnswerRelevancyMetric(
    threshold=0.6,
    model=model,
    include_reason=False
)

faithfulness_metric = FaithfulnessMetric(
    threshold=0.6,
    model=model,
    include_reason=False
)

contextual_relevancy_metric = ContextualRelevancyMetric(
    threshold=0.6,
    model=model,
    include_reason=False
)

hallucination_metric = HallucinationMetric(
    threshold=0.5,
    model=model,
    include_reason=False
)

evaluations = {}

for field, test_case in test_cases_per_field.items():

    evaluation = evaluate(
        test_case,
        [
            answer_relevancy_metric,
            faithfulness_metric,
            contextual_relevancy_metric,
            hallucination_metric,
        ],
    )

    evaluations[field] = evaluation

    with open("medfluencer/evaluation/evaluation_metrcis_rag.json", "w") as file:
        json.dump(evaluations, file)

Output()

Evaluating test cases...
Event loop is already running. Applying nest_asyncio patch to allow async execution...


Output()

Output()

Output()

...

Evaluating test cases...
Event loop is already running. Applying nest_asyncio patch to allow async execution...


Output()

Output()

Output()

RateLimitError: Error code: 429 - {'type': 'error', 'error': {'type': 'rate_limit_error', 'message': 'Number of request tokens has exceeded your daily rate limit (https://docs.anthropic.com/en/api/rate-limits); see the response headers for current usage. Please reduce the prompt length or the maximum tokens requested, or try again later. You may also contact sales at https://www.anthropic.com/contact-sales to discuss your options for a rate limit increase.'}}

In [29]:
evaluations

{'Allgemeinmedizin': 'evaluation',
 'Anästhesiologie': 'evaluation',
 'Anatomie': 'evaluation',
 'Arbeitsmedizin': 'evaluation',
 'Augenheilkunde': 'evaluation',
 'Biochemie': 'evaluation',
 'Chirurgie': 'evaluation',
 'Allgemeine Chirurgie': 'evaluation',
 'Gefäßchirurgie': 'evaluation',
 'Herzchirurgie': 'evaluation',
 'Kinderchirurgie': 'evaluation',
 'Orthopädie und Unfallchirurgie': 'evaluation',
 'Plastische, Rekonstruktive und Ästhetische Chirurgie': 'evaluation',
 'Thoraxchirurgie': 'evaluation',
 'Viszeralchirurgie': 'evaluation',
 'Frauenheilkunde und Geburtshilfe': 'evaluation',
 'Gynäkologische Endokrinologie und Reproduktionsmedizin': 'evaluation',
 'Gynäkologische Onkologie': 'evaluation',
 'Spezielle Geburtshilfe und Perinatalmedizin': 'evaluation',
 'Hals-Nasen-Ohrenheilkunde': 'evaluation',
 'Haut- und Geschlechtskrankheiten': 'evaluation',
 'Humangenetik': 'evaluation',
 'Hygiene und Umweltmedizin': 'evaluation',
 'Innere Medizin': 'evaluation',
 'Angiologie': 'evalua

In [ ]:
evaluation

[TestResult(success=False, metrics_metadata=[MetricMetadata(metric='Answer Relevancy', threshold=0.7, success=True, score=0.8571428571428571, reason='The score is 0.86 because while the answer is relevant and provides some helpful tips on dealing with anxiety disorders, there is a statement in the output that does not contribute any useful information to addressing the question.', strict_mode=False, evaluation_model='gpt-3.5-turbo', error=None, evaluation_cost=0.0014895), MetricMetadata(metric='Faithfulness', threshold=0.7, success=False, score=0.6, reason='The score is 0.60 because the actual output includes contradictions such as focusing on values and goals in life being more important than anxiety, and the idea that relying too much on coping strategies that provide short-term relief is not recommended.', strict_mode=False, evaluation_model='gpt-3.5-turbo', error=None, evaluation_cost=0.0031345), MetricMetadata(metric='Contextual Relevancy', threshold=0.7, success=True, score=0.8, 

TODO: Evaluate different answers for expert or layman (how relevant is the context in each case?)

TODO: Evaluate standard llm answers

TODO: Generate suitable test queries for comments

TODO: test video questions and comment questions on other datasets

TODO: evaluate cobined query engine on both questions


In [ ]:
from llama_index.core import VectorStoreIndex
import numpy as np
import json
from llama_index.core.schema import TextNode

In [ ]:
question = "Was hilft bei Ohrenentzündung?"

In [ ]:
retrieved_nodes = retriever.retrieve(question)

In [ ]:
for node in retrieved_nodes:
    print(node.text)

how do I get rid of pain in my ear pain
can be caused by things like an
infection a blockage in the ear canal or
an injury sometimes you can alleviate
the pain by applying a warm compress
such as putting a warm cloth over the
affected ear to help reduce pain
non-prescription drugs like
acetaminophen can help and even more so
when it comes to NSAID medications so
things like ibuprofen Advil Aleve
naproxen and the like just know that
every medication has the potential for
side effects sometimes ear drops can
help just remember to follow the
directions on the packaging it's best to
keep the hurt ear or the injured ear up
so when you're lying down try to keep
that injured ear facing up to relieve
pressure and swelling there are certain
red flags that you should be aware of
that indicate you may need to seek
urgent medical attention red flags
include severe pain if the pain lasts
for a long time or if it comes with
Associated discharge or hearing loss
antibiotics may be needed for example t

In [ ]:
from llama_index.core.llms import ChatMessage

In [ ]:
summaries = []
for node in retrieved_nodes[:5]:
    res = llm.chat(
        [
            ChatMessage(
                role="system",
                content=f"Summarize the content of the given document which is relevant for the question '{question}'",
            ),
            ChatMessage(role="user", content=node.text),
        ]
    )
    summaries.append(res.message.content)

In [ ]:
summaries

['Hier sind die wichtigsten Punkte, die bei einer Ohrenentzündung helfen können:\n\n- Warme Kompressen auf dem betroffenen Ohr können die Schmerzen lindern.\n\n- Nicht-verschreibungspflichtige Schmerzmittel wie Acetaminophen oder NSAID-Medikamente (Ibuprofen, Naproxen etc.) können die Schmerzen reduzieren.\n\n- Ohrentropfen können ebenfalls hilfreich sein, aber die Anweisungen auf der Packung müssen beachtet werden.\n\n- Das betroffene Ohr nach oben halten, wenn man liegt, um Druck und Schwellungen zu verringern.\n\n- Bei starken anhaltenden Schmerzen, Ausfluss oder Hörverlust sollte dringend ein Arzt aufgesucht werden, da eventuell Antibiotika zur Behandlung einer bakteriellen Infektion nötig sind.\n\nDie Hauptempfehlungen sind also warme Kompressen, Schmerzmittel, Ohrentropfen und das Ohr nach oben halten. Bei Alarmsignalen wie starken Schmerzen oder Komplikationen ist ein Arztbesuch angeraten.',
 'Der gegebene Text behandelt nicht die Frage "Was hilft bei Ohrenentzündung?". Er besch